In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 23 10:33:07 2019
@author: Partha.Dip
"""

from timeit import default_timer as timer
import csv
import random



#enable randomization
enable_random=1
#what percentage of dataset to be used as training set
training=80

#calculate number of total instance and read touples into list
readFile=open('cars_dataset.csv','r')
row_count = sum(1 for row in readFile)
readFile.seek(0,0) #take pinter to beginning
reader = csv.reader(readFile)
lines = list(reader)
readFile.close() 


#exclude the row containing attribute names
row_count=row_count-1

print("Total instances in dataset : ",row_count)

#number of instance in training set
training=int(training*row_count/100)

print("Total instances for training : ",training)

#number of instance in test set
print("Total instances for prediction: ",row_count-training)

testing = row_count-training

#shuffle the lines
if enable_random == 1 :
      random.shuffle(lines)

        
      
"Defining data variables and data structure for avc set"

#variable for number of rows in different classes in testset
unacc_num=0
acc_num=0
vgood_num=0
good_num=0

#variables used calculating probability of different class
unacc_prob=1
acc_prob=1
vgood_prob=1
good_prob=1

#variable for counting how many classifications are correct or wrong
right=0
wrong=0

#Confusion matrix array
cm=[[0 for x in range(4)] for y in range(4)]
#column 0-unacc,1-acc,2-vgood,3-good


#creating avc set structure
# Creates a list containing 4 lists, each of 4 items, all set to 0
column, row = 4, 4;
buying_car = [[0 for x in range(column)] for y in range(row)] 
#row 0-vhigh,1-high,2-med,3-low


column, row = 4, 4;
maint_car = [[0 for x in range(column)] for y in range(row)] 
#row 0-vhigh,1-high,2-med,3-low


column, row = 4, 4;
door_car = [[0 for x in range(column)] for y in range(row)] 
#row 0-two,1-three,2-5more,3-four

column, row = 4, 3;
person_car = [[0 for x in range(column)] for y in range(row)] 
#row 0-two,1-four,2-more

column, row = 4, 3;
lug_boot = [[0 for x in range(column)] for y in range(row)] 
#row 0-small,1-med,2-big 

column, row = 4, 3;
safety = [[0 for x in range(column)] for y in range(row)] 
#row 0-low,1-med,2-high


start=timer() 

'''Avc set population'''


#loop for exploring all the touples
for x in range(1,training+1):
    
    #read the actual class lebel from current touple
    label = lines[x][6]
    
    #defines which column of the avc set is going to be accessed based on
    # class lebel of current touple
    if  label == "unacc":
        column=0
        unacc_num += 1  #increment the counter of class lebel
    elif label == "acc":
        column=1
        acc_num += 1
    elif label == "vgood":
        column=2
        vgood_num += 1
    elif label == "good" :
        column=3
        good_num += 1
        
    
    
                
    """ buying_cars ...... goto column 0 or buying_car attribute column in current line and then 
        compare the valus with predefined ones (vhigh high med low),increment the appropriate
        location in avc set.
    """
    
    if lines[x][0]   == "vhigh" : 
                    buying_car[0][column] += 1
    elif lines[x][0] == "high":
                    buying_car[1][column] += 1
    elif lines[x][0] =="med":
                    buying_car[2][column] += 1
    elif lines[x][0] =="low" :
                    buying_car[3][column] += 1
                    
    #maint_car
                
    """ maint_cars ...... goto column 0 or maint_car attribute column in current line and then 
        compare the valus with predefined ones (vhigh high med low),increment the appropriate
        location in avc set.
    """
    
    if   lines[x][1]==  "vhigh" : 
                    maint_car[0][column]  += 1
    elif lines[x][1] == "high":
                    maint_car[1][column] += 1
    elif lines[x][1]==  "med":
                    maint_car[2][column] += 1
    elif lines[x][1]==  "low" :
                    maint_car[3][column] += 1
                    
    #door_car
    if   lines[x][2] ==  "two" : 
                    door_car[0][column] += 1
    elif lines[x][2] == "three":
                    door_car[1][column] += 1
                
    elif lines[x][2] == "5more":
                    door_car[2][column] += 1
                
    elif lines[x][2] == "four":           
                    door_car[3][column] += 1
    
                    
    #person
    if lines[x][3]   == "two" : 
                    person_car[0][column] += 1
    elif lines[x][3] == "four":
                    person_car[1][column] += 1
                
    elif lines[x][3] == "more":
                    person_car[2][column] += 1
                    
    #lug_boot
    if lines[x][4]   == "small" : 
                    lug_boot[0][column] += 1
    elif lines[x][4] == "med":
                    lug_boot[1][column] += 1
                
    elif lines[x][4] == "big":
                    lug_boot[2][column] += 1
                    
    #safety
    if lines[x][5]   == "low" : 
                   safety[0][column] += 1
    elif lines[x][5] == "med":
                   safety[1][column] += 1
                
    elif lines[x][5] == "high":
                   safety[2][column] += 1
            
end=timer()   
train_time=end-start
   
start=timer()   
#print(unacc_num+acc_num+vgood_num+good_num)          
""" Classification """

""" loop for exploring training set touples """
for x in range(training+1,row_count+1):
        unacc_prob=1
        acc_prob=1
        vgood_prob=1
        good_prob=1
        #variable for laplacian correction
        corr=0
        #loop for exploring all columns in current line 
        for y in range(6):
               
             if y==0: #when in buying_car position of a line
                 if   lines[x][0]== "vhigh" : 
                    row=0 #which row of the avc set is going to be accessed
                 elif lines[x][0] == "high":
                    row=1
                 elif lines[x][0]=="med":
                    row=2
                 elif lines[x][0]=="low":
                    row=3
                 #add corr=4 as distinct attribute here is 4
                 corr=4
                 a=buying_car[row][0] #unacc column and matched row
                 b=buying_car[row][1] #acc column and mattched row
                 c=buying_car[row][2] #vgood
                 d=buying_car[row][3]
                 
             elif y==1: #maint_car position of the line
                 if   lines[x][1] == "vhigh" : 
                    row=0
                 elif lines[x][1] == "high":
                    row=1
                 elif lines[x][1] =="med":
                    row=2
                 elif lines[x][1] =="low":
                    row=3
                 corr=4
                 a=maint_car[row][0]
                 b=maint_car[row][1]
                 c=maint_car[row][2]
                 d=maint_car[row][3]
                
             elif y==2: #doors
                 if   lines[x][2] == "two" : 
                    row=0
                 elif lines[x][2] == "three":
                    row=1
                 
                 elif lines[x][2] == "5more":
                    row=2
                 elif lines[x][2] == "four":
                    row=3
                    
                 corr=4 
                 a=door_car[row][0]
                 b=door_car[row][1]
                 c=door_car[row][2]
                 d=door_car[row][3]
                    
             elif  y==3: #persons
                 if   lines[x][3] == "two" : 
                    row=0
                 elif lines[x][3] == "four":
                    row=1
                 
                 elif lines[x][3] == "more":
                    row=2
                 corr=3
                 a=person_car[row][0]
                 b=person_car[row][1]
                 c=person_car[row][2]
                 d=person_car[row][3]
                    
             elif  y==4: #lug_boot
                 if   lines[x][4] == "small" : 
                    row=0
                 elif lines[x][4] == "med":
                    row=1
                 
                 elif lines[x][4] == "big":
                    row=2
                 corr=3
                 a=lug_boot[row][0]
                 b=lug_boot[row][1]
                 c=lug_boot[row][2]
                 d=lug_boot[row][3]
                 
             elif y==5: #safety
                 if   lines[x][5] == "low" : 
                    row=0
                 elif lines[x][5] == "med":
                    row=1
                 
                 elif lines[x][5] == "high":
                    row=2
                 corr=3
                 a=safety[row][0]
                 b=safety[row][1]
                 c=safety[row][2]
                 d=safety[row][3]
                    
                   
             if a == 0:   #unacc column 
                          unacc_prob = unacc_prob * 1/(unacc_num+corr) 
                          
             else :       
                          unacc_prob = unacc_prob * a/unacc_num
             if b == 0:
                          acc_prob = acc_prob * 1/(acc_num+corr)
                          
             else :       
                          acc_prob = acc_prob * b/acc_num
             
             if c == 0:
                          vgood_prob = vgood_prob * 1/(vgood_num+corr)
                           
             else :       
                          vgood_prob = vgood_prob * c/vgood_num
                 
             if d == 0:
                          good_prob = good_prob * 1/(good_num+corr)
                          
                           
             else :       
                           good_prob = good_prob * d/good_num
                           
                           
        #as we explore the columns of a line conditional probability is
        #calculated and multiplied now just multiply prior probability                 
        unacc_prob=unacc_prob*unacc_num/training
        predict="unacc" #by default set as the predicted class for current line 
        val=unacc_prob  #val contains the probability of prected class
        acc_prob=acc_prob*acc_num/training
        if acc_prob > val: #if current probability better than previous one, update
            predict="acc"
            val=acc_prob
        vgood_prob=vgood_prob*vgood_num/training
        if vgood_prob > val:
            predict="vgood"
            val=vgood_prob
        good_prob=good_prob*good_num/training
        if  good_prob > val:
            predict="good"
            
         ##enter value into confusion matrix
         
        if    lines[x][6] == "unacc" : #read actual class from the lines class lebel column and define the
                                       #row in which we have to update
         row=0
        elif  lines[x][6] == "acc" :
         row=1
        elif  lines[x][6] =="vgood":
         row=2
        elif  lines[x][6] =="good":
         row=3
        """Now the predict contains the predicted class now predic is
        compared with different class lebels and value is update in appropriate cell 
        in confusion matrix
        """
        if predict=="unacc": 
            cm[row][0]+= 1
        elif predict=="acc":
            cm[row][1]+= 1
        elif predict=="vgood":
            cm[row][2]+= 1
        elif predict=="good":
            cm[row][3]+= 1
            
        """ calculate how many predictions are right and wrong"""
        if lines[x][6] == predict :# if what was predicted and what is actual class label is same
            right+= 1
        else:
            wrong+= 1
end=timer()         
test_time=end-start
        
"Printing results"


#print number of right and wrong classification
print("Correctly classified instances: ",right)   
print("Inorrectly classified instances: ",wrong,"\n") 
                
#print confusion matrix

actual=['unacc','acc  ','vgood','good ']

print("-------Confusion Matrix-------")
print("      unacc  acc  vgood  good")
for x in range(4)  :
  print(actual[x]," ",end="")
  for y in range(4) :    
          
          print(cm[x][y] , end="") 
          print("    ",end="")
  print("\n")
#print("a-unacc,b-acc,c-vgood,d-good\n") 

#print the accuracy of the model
print("Accuracy of the model = ", right*100/(right+wrong) ,"%\n")

#calculate recall,precision and F1

for x in range (4):
    recaldiv=0 #divisor for recall
    precisiondiv=0 #divisor for precision
    fpup=0 #numerator for false positive
    fpdown= 0#denominator for fp
    
    
    for y in range (4) :
        recaldiv=recaldiv+cm[x][y] #add all element of the row 
        precisiondiv=precisiondiv+cm[y][x] #add all element of column
        if x!=y : #all element that has been predicted as the calss but is not,not in x,x position
            fpup=fpup+cm[y][x] 
            
            
        
    fpdown=testing-recaldiv #recalldiv is actual in class,testing-recaldiv=all that not in class
                             
    
    if cm[x][x] == 0 : 
        recall=0
        if precisiondiv == 0:
            prec= "?"
        else:
            prec=0
            
        f1="?"
          
    else: 
        
        recall=cm[x][x]/recaldiv
        fprate=fpup/fpdown
        prec=cm[x][x]/precisiondiv
        f1=2*(recall*prec)/(recall+prec)
    
    print("Recall for class : ",actual[x],"= ",recall)
    print("FP rate for class : ",actual[x],"= ",fprate)
    
    print("Precision for class : ",actual[x],"= ",prec)
    print("F1 for class : ",actual[x],"= ",f1)
    print("\n")
          
   
print("Training Time ",train_time)
print("Testing Time ",test_time)
#print(unacc_num+acc_num+vgood_num+good_num)      


Total instances in dataset :  1728
Total instances for training :  1382
Total instances for prediction:  346
Correctly classified instances:  299
Inorrectly classified instances:  47 

-------Confusion Matrix-------
      unacc  acc  vgood  good
unacc  228    6    0    0    

acc    27    57    0    4    

vgood  0    2    9    0    

good   0    6    2    5    

Accuracy of the model =  86.41618497109826 %

Recall for class :  unacc =  0.9743589743589743
FP rate for class :  unacc =  0.24107142857142858
Precision for class :  unacc =  0.8941176470588236
F1 for class :  unacc =  0.9325153374233129


Recall for class :  acc   =  0.6477272727272727
FP rate for class :  acc   =  0.05426356589147287
Precision for class :  acc   =  0.8028169014084507
F1 for class :  acc   =  0.7169811320754716


Recall for class :  vgood =  0.8181818181818182
FP rate for class :  vgood =  0.005970149253731343
Precision for class :  vgood =  0.8181818181818182
F1 for class :  vgood =  0.8181818181818182


Re